In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data= pd.read_csv("../input/usa-housing/USA_Housing.csv")
df= data.copy()
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().values.any()

Adres bilgisi gereksiz olduğunu düşündüğüm için siliyorum

In [ ]:
df.drop("Address", axis=1, inplace= True)
df.head()

Price değişkeni ile aralarındaki korelasyona bakarak en yüksek korelasyona sahip değişkeni seçiyorum

In [ ]:
df.corr()

# Basit Doğrusal Regresyon

In [ ]:
from sklearn.linear_model import LinearRegression

Korelasyonu yüksek olan değişkeni kullanarak basit doğrusal regresyon modeli oluşturuyorum

In [ ]:
x = df[["Avg. Area Income"]]
y = df["Price"]

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

Modele giriş ve çıkış olarak verilecek verileri eğitim ve test seti olarak ayırıyorum

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state= 28)

In [ ]:
reg = LinearRegression()
model = reg.fit(x_train, y_train)
model.intercept_ , model.coef_

Elde edilen sabit değer ve katsayıları kullanarak tahmin fonksiyonunu oluşturuyorum

Price= -225334.13 + 21.23*Avg. Area Income

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

Model performansını değerlendirmek için hatayı hesaplıyorum

In [ ]:
mse= mean_squared_error(y_test,model.predict(x_test))
rmse= np.sqrt(mse)
print("MSE: {}\nRMSE: {}".format(mse,rmse))

# Çoklu Doğrusal Regresyon

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [ ]:
df.head()

In [ ]:
x= df.drop("Price", axis=1)
y= df["Price"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state= 22)

In [ ]:
x_train.shape , x_test.shape , y_train.shape , y_test.shape

In [ ]:
lm = LinearRegression()
model = lm.fit(x_train, y_train)

In [ ]:
print("Sabit Değer: {} \nKatsayılar: {}".format(model.intercept_, model.coef_))

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, model.predict(x_test)))
print("Test seti RMSE: {}".format(rmse))

## Model Tuning

In [ ]:
df.head()

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
x_train.shape , x_test.shape , y_train.shape , y_test.shape

In [ ]:
model

In [ ]:
cross_val_score(model, x_train, y_train, cv = 10, scoring = "r2")

In [ ]:
t_rmse=np.sqrt(-cross_val_score(model, 
                x_train, 
                y_train, 
                cv = 10, 
                scoring = "neg_mean_squared_error")).mean()
print("Tune Edilmiş Eğitim seti RMSE: {}".format(t_rmse))

In [ ]:
t_rmse= np.sqrt(-cross_val_score(model, 
                x_test, 
                y_test, 
                cv = 10, 
                scoring = "neg_mean_squared_error")).mean()
print("Tune Edilmiş Test seti RMSE: {}".format(t_rmse))

# PCR Regresyon

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale 
pca = PCA()

Eğitim seti üzerinde indirgeme işlemi gerçekleştiriyorum

In [ ]:
x_reduced_train = pca.fit_transform(scale(x_train)) 

In [ ]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals = 4)*100)

In [ ]:
lm = LinearRegression()

In [ ]:
pcr_model = lm.fit(x_reduced_train, y_train)

In [ ]:
print("Sabit Değer: {}\nKatsayılar: {}".format(pcr_model.intercept_, pcr_model.coef_))

In [ ]:
y_pred = pcr_model.predict(x_reduced_train)

In [ ]:
np.sqrt(mean_squared_error(y_train, y_pred))

In [ ]:
r2_score(y_train, y_pred)

In [ ]:
pca2 = PCA()
x_reduced_test = pca2.fit_transform(scale(x_test))
y_pred = pcr_model.predict(x_reduced_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

## Model Tuning

In [ ]:
from sklearn import model_selection
cv_10 = model_selection.KFold(n_splits = 10,
                             shuffle = True,
                             random_state = 42)

In [ ]:
lm = LinearRegression()
RMSE = []

In [ ]:
for i in np.arange(1, x_reduced_train.shape[1] + 1):
    
    score = np.sqrt(-1*model_selection.cross_val_score(lm, 
                                                       x_reduced_train[:,:i], 
                                                       y_train.ravel(), 
                                                       cv=cv_10, 
                                                       scoring='neg_mean_squared_error').mean())
    RMSE.append(score)
    
RMSE

minimum hata için tüm değişkenleri almamız gerekiyor.

# PLS Regresyon

In [ ]:
from sklearn.cross_decomposition import PLSRegression, PLSSVD

In [ ]:
pls_model = PLSRegression().fit(x_train, y_train)

In [ ]:
pls_model.coef_

In [ ]:
y_pred = pls_model.predict(x_train)

In [ ]:
np.sqrt(mean_squared_error(y_train, y_pred))

In [ ]:
r2_score(y_train, y_pred)

In [ ]:
y_pred = pls_model.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

## Model Tuning

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#CV
cv_10 = model_selection.KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
#Hata hesaplamak için döngü
RMSE = []

for i in np.arange(1, x_train.shape[1] + 1):
    pls = PLSRegression(n_components=i)
    score = np.sqrt(-1*cross_val_score(pls, x_train, y_train, cv=cv_10, scoring='neg_mean_squared_error').mean())
    RMSE.append(score)

Sonuçların Görselleştirilmesi

In [ ]:
plt.plot(np.arange(1, x_train.shape[1] + 1), np.array(RMSE), '-v', c = "r")
plt.xlabel('Bileşen Sayısı')
plt.ylabel('RMSE')
plt.title('Salary');

In [ ]:
pls_model = PLSRegression(n_components = 5).fit(x_train, y_train)

In [ ]:
y_pred = pls_model.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

# Ridge Regresyon

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
lambda_  = np.random.rand()
ridgeModel= Ridge(alpha= lambda_).fit(x_train,y_train)
lambda_

In [ ]:
ridgeModel.coef_

In [ ]:
lambdalar = 10**np.linspace(10,-2,10)*0.5 

In [ ]:
ridge_model = Ridge()
katsayilar = []

for i in lambdalar:
    ridge_model.set_params(alpha = i)
    ridge_model.fit(x_train, y_train) 
    katsayilar.append(ridge_model.coef_) 
    

    
ax = plt.gca()
ax.plot(lambdalar, katsayilar) 
ax.set_xscale('log') 

plt.xlabel('Lambda(Alpha) Değerleri')
plt.ylabel('Katsayılar/Ağırlıklar')
plt.title('Düzenlileştirmenin Bir Fonksiyonu Olarak Ridge Katsayıları');

In [ ]:
y_pred= ridgeModel.predict(x_train)

In [ ]:
np.sqrt(mean_squared_error(y_train,y_pred))

In [ ]:
y_pred= ridgeModel.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

## Model Tuning

In [ ]:
lambdalar

In [ ]:
from sklearn.linear_model import RidgeCV
ridge_cv = RidgeCV(alphas = lambdalar, 
                   scoring = "neg_mean_squared_error",
                   normalize = True)

In [ ]:
ridge_cv.fit(x_train,y_train);

In [ ]:
ridge_cv.alpha_

In [ ]:
ridge_tuned = Ridge(alpha = ridge_cv.alpha_, 
                   normalize = True).fit(x_train,y_train)

In [ ]:
ridge_tuned.coef_

In [ ]:
np.sqrt(mean_squared_error(y_train,ridge_tuned.predict(x_train)))

In [ ]:
np.sqrt(mean_squared_error(y_test,ridge_tuned.predict(x_test)))

# Lasso Regresyon

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso_model= Lasso().fit(x_train,y_train)

In [ ]:
y_pred= lasso_model.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

## Model Tuning

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
lasso_cv_model = LassoCV(alphas = None, 
                         cv = 10, 
                         max_iter = 1000, 
                         normalize = True)

In [ ]:
lasso_cv_model.fit(x_train,y_train)

In [ ]:
lasso_cv_model.alpha_

In [ ]:
lasso_tuned= Lasso(alpha= lasso_cv_model.alpha_)

In [ ]:
lasso_tuned.fit(x_train,y_train)

In [ ]:
y_pred= lasso_tuned.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

# ElasticNet Regresyon

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
enet_model= ElasticNet().fit(x_train,y_train)

In [ ]:
enet_model.coef_

In [ ]:
enet_model.intercept_

In [ ]:
y_pred= enet_model.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

## Model Tuning

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
enet_cv_model= ElasticNetCV(cv= 10, random_state= 42 ).fit(x_train,y_train)

In [ ]:
enet_cv_model.alpha_

In [ ]:
enet_tuned= ElasticNet(alpha= enet_cv_model.alpha_).fit(x_train,y_train)

In [ ]:
y_pred= enet_tuned.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
r2_score(y_test,y_pred)